In [1]:
!pip install -q condacolab


✨🍰✨ Everything looks OK!


In [2]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [3]:
!conda install -c pytorch faiss-cpu -y

Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [3]:
!pip install -r requirements.txt

In [2]:
import sys
# If faiss is not found after conda install, try pip install
try:
    import faiss
except ModuleNotFoundError:
    print("faiss not found, attempting pip install...")
    !{sys.executable} -m pip install faiss-cpu
    import faiss

print("FAISS:", faiss.__version__)

faiss not found, attempting pip install...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 62.0 MB/s eta 0:00:00
FAISS: 1.13.2


In [5]:
import json
import numpy as np
import faiss
import os
from sentence_transformers import SentenceTransformer

# --- TOGGLE THIS ---
DRY_RUN = False  # Set to True for 3 records, False for all 5000+
# -------------------

# 1. Configuration
MODEL_NAME = 'paraphrase-multilingual-MiniLM-L12-v2'
INPUT_FILE = "/content/articles_with_ner.jsonl"
INDEX_PATH = "vector_index.faiss"
METADATA_PATH = "metadata.json"

# 2. Load Model
print(f"Loading Model {MODEL_NAME}...")
model = SentenceTransformer(MODEL_NAME)

def build_index():
    documents = []
    metadata = []

    print(f"Reading JSONL (DRY_RUN is {DRY_RUN})...")

    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            # If DRY_RUN is True, stop after 3 records.
            # If False, loop through all 5000+.
            if DRY_RUN and i >= 3:
                break

            item = json.loads(line)
            # Combine Title + Body
            full_text = f"{item.get('title', '')} {item.get('body', '')}"
            documents.append(full_text)

            # Keep metadata to display results later
            metadata.append({
                "id": i,
                "title": item.get("title"),
                "url": item.get("url"),
                "source": item.get("source")
            })

    print(f"Generating embeddings for {len(documents)} records...")
    # This encodes the text into vectors
    embeddings = model.encode(documents, show_progress_bar=True, convert_to_numpy=True)

    # 3. Create FAISS Index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    faiss.normalize_L2(embeddings)
    index.add(embeddings)

    # 4. Save everything
    faiss.write_index(index, INDEX_PATH)
    with open(METADATA_PATH, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)

    print(f"✅ Done! Files saved: {INDEX_PATH}, {METADATA_PATH}")

if __name__ == "__main__":
    build_index()

Loading Model paraphrase-multilingual-MiniLM-L12-v2...
Reading JSONL (DRY_RUN is False)...
Generating embeddings for 5062 records...


Batches:   0%|          | 0/159 [00:00<?, ?it/s]

✅ Done! Files saved: vector_index.faiss, metadata.json


In [6]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer

# 1. Load the Index and Metadata
index = faiss.read_index("vector_index.faiss")
with open("metadata.json", "r", encoding='utf-8') as f:
    meta = json.load(f)

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 2. Define Entity Map (Task 5)
# In a real run, this would be mined from your articles_with_ner.jsonl
entity_map = {
    "bnp": "বিএনপি",
    "tarek": "তারেক রহমান",
    "nahid": "নাহিদ ইসলাম"
}

def search_test(user_query):
    print(f"\n--- Testing Query: '{user_query}' ---")

    # Task 4 & 5: Query Expansion / Entity Mapping
    processed_query = user_query.lower()
    for en, bn in entity_map.items():
        if en in processed_query:
            processed_query += f" {bn}"
            print(f"Mapped Entity: {en} -> {bn}")

    # Convert to Vector
    vec = model.encode([processed_query])
    faiss.normalize_L2(vec)

    # Search (k=1 means get the top result)
    distances, indices = index.search(vec, k=1)

    match_idx = indices[0][0]
    score = distances[0][0]

    print(f"Match Found: {meta[match_idx]['title']}")
    print(f"Similarity Score: {score:.4f}")
    print(f"Source: {meta[match_idx]['source']}")

# --- Run Dry Run Tests ---

# Test 1: Search for the Prothom Alo Opinion piece (Nahid Islam)
search_test("wealth of Nahid Islam")

# Test 2: Search for the weapons recovery article
search_test("police weapons recovered in Araihazar")

# Test 3: Search using an Entity Map trigger
search_test("latest news about Tarek Rahman")


--- Testing Query: 'wealth of Nahid Islam' ---
Mapped Entity: nahid -> নাহিদ ইসলাম
Match Found: ২৫ প্রার্থীর মধ্যে আয়-সম্পদে এগিয়ে বিএনপির আশরাফ
Similarity Score: 0.6670
Source: prothom_alo

--- Testing Query: 'police weapons recovered in Araihazar' ---
Match Found: 2 looted firearms recovered, 2 arrested in Chandpur
Similarity Score: 0.6624
Source: newage

--- Testing Query: 'latest news about Tarek Rahman' ---
Mapped Entity: tarek -> তারেক রহমান
Match Found: তারেক রহমানের সিলেট সফর চূড়ান্ত, তিন জেলায় নির্বাচনী সমাবেশ
Similarity Score: 0.5806
Source: prothom_alo
